In [1003]:
# Standard library imports
import os
import sys
import re
import warnings
import random
import hashlib

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning and preprocessing
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  # Assuming you might need it

# Specific models and tools
from xgboost import XGBClassifier
import xgboost as xgb

# Encoding and feature selection
from category_encoders import TargetEncoder  # Fixed the import based on usage
from scipy.stats import randint, uniform

# Model persistence
from joblib import dump, load

# Miscellaneous settings
%matplotlib inline
warnings.filterwarnings('ignore')


In [1004]:
comps = [
    'E0', 
    'E1', 'E2', 'E3',  'SC0', 'SC1',    'D1',     'D2',    'F1',     'F2',    'I1',     'I2',    'SP1',    'SP2',    'B1',    'G1',    'N1',    'P1',    'T1',
]

seasons = [
    '2324', 
    '2223', 
    '2122', 
    '2021',
    #'1920', 
    #'1819', 
    #'1718', 
    #'1617',
    #'1516', '1415', 
    #'1314', '1213',
    #'1112', '1011', 
    #'0910', 
    #'0809',
    #'0708', '0607', '0506', '0405',
    #'0304', '0203', '0102', '0001',
]

fixtures = [
    "fixtures",
    #"new_league_fixtures"
]


In [1005]:
# Set the dataprep_start_date to the date the data preparation should start
# If None, the data preparation will start from the beginning of the data

# Make sure the file below already exists if you want to start from a specific date
# file should be in the format "processed_data_<content>.csv"
content = "euro_4s"


dataprep_start_date = None
#dataprep_start_date = pd.Timestamp(year=2024, month=4, day=18)

In [1006]:
# Load all filepaths into a list
matches_files = []
fixtures_files = []

In [1007]:
for season in seasons:    
    for comp in comps:  
        matches_files.append('data/scraped/%s/%s.csv' % (season, comp))
        continue

In [1008]:
for fixture in fixtures:    
    fixtures_files.append(f'data/scraped/{seasons[0]}/{fixture}.csv')
    continue

In [1009]:
# Load and concatenate matches data into a single DataFrame
df = pd.DataFrame()

for file in matches_files:

    try:

        year = re.search(r'(\d{4})', file).group(1)
        print(f'Loading {file}')

        df_temp = pd.read_csv(file)

        # add the year to the dataframe as a column 'Season'
        df_temp['Season'] = year

        df = pd.concat([df, df_temp], ignore_index=True)
    except:
        # print an error message
        print(f'Error: {file} not found')

# print the amount of data loaded
print(f"Data loaded: {df.shape[0]} matches")

Loading data/scraped/2324/E0.csv
Loading data/scraped/2324/E1.csv
Loading data/scraped/2324/E2.csv
Loading data/scraped/2324/E3.csv
Loading data/scraped/2324/SC0.csv
Loading data/scraped/2324/SC1.csv
Loading data/scraped/2324/D1.csv
Loading data/scraped/2324/D2.csv
Loading data/scraped/2324/F1.csv
Loading data/scraped/2324/F2.csv
Loading data/scraped/2324/I1.csv


Loading data/scraped/2324/I2.csv
Loading data/scraped/2324/SP1.csv
Loading data/scraped/2324/SP2.csv
Loading data/scraped/2324/B1.csv
Loading data/scraped/2324/G1.csv
Loading data/scraped/2324/N1.csv
Loading data/scraped/2324/P1.csv
Loading data/scraped/2324/T1.csv
Loading data/scraped/2223/E0.csv
Loading data/scraped/2223/E1.csv
Loading data/scraped/2223/E2.csv
Loading data/scraped/2223/E3.csv
Loading data/scraped/2223/SC0.csv
Loading data/scraped/2223/SC1.csv
Loading data/scraped/2223/D1.csv
Loading data/scraped/2223/D2.csv
Loading data/scraped/2223/F1.csv
Loading data/scraped/2223/F2.csv
Loading data/scraped/2223/I1.csv
Loading data/scraped/2223/I2.csv
Loading data/scraped/2223/SP1.csv
Loading data/scraped/2223/SP2.csv
Loading data/scraped/2223/B1.csv
Loading data/scraped/2223/G1.csv
Loading data/scraped/2223/N1.csv
Loading data/scraped/2223/P1.csv
Loading data/scraped/2223/T1.csv
Loading data/scraped/2122/E0.csv
Loading data/scraped/2122/E1.csv
Loading data/scraped/2122/E2.csv
Load

In [1010]:
# Load and concatenate matches data into a single DataFrame
df_fixtures = pd.DataFrame()

for file in fixtures_files:

    try:

        year = re.search(r'(\d{4})', file).group(1)
        print(f'Loading {file}')

        df_temp = pd.read_csv(file)

        # add the year to the dataframe as a column 'Season'
        df_temp['Season'] = year

        df_fixtures = pd.concat([df_fixtures, df_temp], ignore_index=True)
    except:
        # print an error message
        print(f'Error: {file} not found')

# print the amount of data loaded
print(f"Data loaded: {df_fixtures.shape[0]} fixtures")

Loading data/scraped/2324/fixtures.csv
Data loaded: 181 fixtures


In [1011]:
def parse_date_to_int(date_str):
    # Split the date_str by the "/" character into day, month, year
    components = date_str.split('/')
    
    # If split was successful but not in expected format, try splitting by absence of separator for '%d%m%Y' or '%d%m%y'
    if len(components) == 1:
        if len(date_str) in [6, 8]:  # Length 6 for '%d%m%y', 8 for '%d%m%Y'
            day, month = int(date_str[:2]), int(date_str[2:4])
            year = int(date_str[4:])
        else:
            return 19000101  # Return default if format does not match expected
    else:
        day, month = int(components[0]), int(components[1])
        year = int(components[2])
    
    # Adjust the year if it was only 2 characters long
    if year < 100:
        year += 2000
    
    # Create a date variable by using the day, month, year integers
    # Note: Direct creation of date variable skipped to avoid unnecessary complexity,
    # directly formatting to YYYYMMDD integer format instead.
    date_int = int(f"{year:04d}{month:02d}{day:02d}")
    
    return date_int

In [1012]:
# Parse the 'Date' column to a datetime object
df_fixtures['Date_temp'] = pd.to_datetime(df_fixtures['Date'], format='%d/%m/%Y')

# Convert the datetime object to an integer in the format YYYYMMDD
df_fixtures['Date_temp'] = df_fixtures['Date_temp'].apply(
    lambda x: int(x.strftime('%Y%m%d')) if pd.notnull(x) else 19000101
)


In [1013]:
# Replace all values with -1 in FTR column
df_fixtures['FTR'].fillna('X', inplace=True)

In [1014]:
# Find the lowest fixture date
# This is the date where the data preparation will start
fixture_cutoff = df_fixtures['Date'].min()

In [1015]:
fixture_cutoff

'19/04/2024'

In [1016]:
# Remove all the rows in df that are after the fixture_cutoff date
df = df[df['Date'] < fixture_cutoff]

In [1017]:
# Concatenate the matches and fixtures dataframes
df = pd.concat([df, df_fixtures], ignore_index=True)

In [1018]:
# Check for duplicate column names
print(df.columns[df.columns.duplicated()])

Index([], dtype='object')


In [1019]:
# Remove all the rows in the dataframe where the 'Div' is not in the list of comps
df = df[df['Div'].isin(comps)]

In [1020]:
# Convert 'Div' to a categorical type, a numeric representation of the division
df['Div'] = df['Div'].astype('category').cat.codes

In [1021]:
file_path = f"data/teams_dict_{content}.txt"

# Check if the file exists
if os.path.exists(file_path):
    # Load the dictionary from the file
    with open(file_path, 'r') as file:
        teams_dict = eval(file.read())  # Using eval to convert string back to dictionary
    # Find the maximum index currently in the dictionary
    max_index = max(teams_dict.values())

    print(f"max index: {max_index}")
else:
    teams_dict = {}
    max_index = -1  # Start from -1 so the first new index will be 0

# Get all teams from DataFrame
all_teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna().unique()
all_teams.sort()

# Create a dictionary of new teams alone
new_teams = {team: index for index, team in enumerate(all_teams) if team not in teams_dict}

# Update dictionary only with new teams, starting indices from max_index + 1
start_index = max_index + 1
teams_dict.update({team: index + start_index for index, team in enumerate(new_teams) if team not in teams_dict})

# Save the updated dictionary to a file
with open(file_path, 'w') as file:
    file.write(str(teams_dict))

# Add team ID columns to DataFrame
df['Team_ID'] = df['HomeTeam'].map(teams_dict)
df['Opp_ID'] = df['AwayTeam'].map(teams_dict)

In [1022]:
def clean_duplicates(df):
    # Sort the DataFrame so that rows with 'FTR' == -1 come first
    df.sort_values(by=['Date', 'Team_ID', 'Opp_ID', 'FTR'], ascending=[True, True, True, False], inplace=True)
    
    # Drop duplicates based on 'Date', 'Team_ID', and 'Opp_ID' keeping the first occurrence (where 'FTR' is -1)
    df = df.drop_duplicates(subset=['Date', 'Team_ID', 'Opp_ID'], keep='first')
    
    return df

df = clean_duplicates(df)

In [1023]:
df['FTR'].value_counts()

FTR
H    6999
A    5079
D    4296
X     159
Name: count, dtype: int64

In [1024]:
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Season,Unnamed: 105,Date_temp,Team_ID,Opp_ID
12448,3,01/01/2021,17:30,Everton,West Ham,0.0,1.0,A,0.0,0.0,...,1.94,2.00,2.16,1.93,1.95,2021,NaN,NaN,136,417
12449,3,01/01/2021,20:00,Man United,Aston Villa,2.0,1.0,H,1.0,0.0,...,1.97,2.07,2.08,1.90,1.98,2021,NaN,NaN,243,34
12720,4,01/01/2021,17:30,Sheffield Weds,Derby,1.0,0.0,H,0.0,0.0,...,1.92,2.01,2.05,1.92,1.92,2021,NaN,NaN,349,115
8308,3,01/01/2022,12:30,Arsenal,Man City,1.0,2.0,A,1.0,0.0,...,1.87,2.09,1.89,2.06,1.84,2122,NaN,NaN,31,242
9258,6,01/01/2022,15:00,Barrow,Bradford,1.0,2.0,A,0.0,1.0,...,1.96,1.97,1.98,1.89,1.93,2122,NaN,NaN,45,64


### Feature Engineering

In [1025]:
# Calculate ELO ratings for each team

# Initialize ratings dictionary
teams = pd.concat([df['Team_ID'], df['Opp_ID']]).unique()
ratings = {team: 1500 for team in teams}

def calculate_expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

def update_elo(rating, actual_score, expected_score, k=30):

    rating = rating + k * (actual_score - expected_score)

    # Parse the rating as an integer with no decimal points
    return int(rating)

# Iterate over the DataFrame and update ELO ratings after each match
elo_team = []
elo_opp = []

for index, row in df.iterrows():
    home_team, away_team, home_score, away_score = row['Team_ID'], row['Opp_ID'], row['FTHG'], row['FTAG']
    home_rating = ratings[home_team]
    away_rating = ratings[away_team]
    
    # Calculate expected scores
    expected_home = calculate_expected_score(home_rating, away_rating)
    expected_away = calculate_expected_score(away_rating, home_rating)
    
    # Calculate actual scores
    actual_home = 1 if home_score > away_score else 0.5 if home_score == away_score else 0
    actual_away = 1 - actual_home
    
    # Update ratings
    new_home_rating = update_elo(home_rating, actual_home, expected_home)
    new_away_rating = update_elo(away_rating, actual_away, expected_away)
    
    # Store updated ratings in the ratings dictionary
    ratings[home_team] = new_home_rating
    ratings[away_team] = new_away_rating
    
    # Append current ratings to list
    elo_team.append(new_home_rating)
    elo_opp.append(new_away_rating)


In [1026]:
# Assign new ELO ratings to the DataFrame
df['team_elo'] = elo_team
df['opp_elo'] = elo_opp

In [1027]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)

# Apply the modified function
df['Date_temp'] = df['Date'].apply(lambda x: parse_date_to_int(x.strftime('%d/%m/%Y')) if pd.notnull(x) else 19000101)

# Day of the week as an integer
df['DayOTW'] = df['Date'].dt.dayofweek

df['Time'] = df['Time'].fillna('00:00').str.replace(':', '').astype(int)

# Only keep the first 2 digits of the Time column, no decimals
df['Time'] = df['Time'] // 100

# Sort df by Date_temp and Time
df = df.sort_values(['Date_temp', 'Time'])

In [1028]:
df.columns = [re.sub(r'[<]', '_st_', str(col)) for col in df.columns]
df.columns = [re.sub(r'[>]', '_gt_', str(col)) for col in df.columns]

In [1029]:
def points(df, row, team_column):
    # Initialize points
    total_points = 0

    # Season of the current match
    current_season = row['Season']

    # Date of the current match
    current_date = row['Date']

    # Define the opponent column based on the team column
    if team_column == 'Team_ID':
        home_team_col = 'Team_ID'
        away_team_col = 'Opp_ID'
    else:
        home_team_col = 'Opp_ID'
        away_team_col = 'Team_ID'

    # Filter DataFrame for matches from the same season before the current date
    past_matches = df[
        (df['Season'] == current_season) & 
        (df['Date'] < current_date) &
        ((df[home_team_col] == row[team_column]) | (df[away_team_col] == row[team_column]))
    ]

    # Calculate points based on the results
    for match in past_matches.itertuples():
        if (getattr(match, home_team_col) == row[team_column] and getattr(match, 'FTR') == 'H') or \
           (getattr(match, away_team_col) == row[team_column] and getattr(match, 'FTR') == 'A'):
            total_points += 3  # Win
        elif getattr(match, 'FTR') == 'D':
            total_points += 1  # Draw

    total_points = round(total_points, 0)

    return total_points


In [1030]:
df['team_points'] = df.apply(lambda x: points(df, x, 'Team_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)
df['opp_points'] = df.apply(lambda x: points(df, x, 'Opp_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)

In [1031]:
def history_vs_opponent_weighted(df, row, team_column):
    # Determine opponent column based on team column
    opponent_column = 'Team_ID' if team_column == 'Opp_ID' else 'Opp_ID'

    # Combine year, month, and day into an integer 'Date_temp'
    row_date_temp = row['Date'].year * 10000 + row['Date'].month * 100 + row['Date'].day

    # Filter for matches between specified teams, excluding current match
    mask = (
        ((df[team_column] == row[team_column]) & (df[opponent_column] == row[opponent_column])) |
        ((df[team_column] == row[opponent_column]) & (df[opponent_column] == row[team_column]))
    ) & (df['Date_temp'] < row_date_temp)

    filtered_matches = df[mask]
    
    if filtered_matches.empty:
        return 0  # Return early if no matches found

    # Sort by date and select top 5 recent matches
    recent_matches = filtered_matches.sort_values(by='Date', ascending=False).head(5)
    weights = list(range(len(recent_matches), 0, -1))  # Descending weights

    # Calculate weighted score based on match results
    weighted_score = sum(
        (3 * weight if match.FTR == 'H' and match.__getattribute__(team_column) == match.Team_ID or
                      match.FTR == 'A' and match.__getattribute__(team_column) != match.Team_ID else
         1 * weight if match.FTR == 'D' else 0)
        for match, weight in zip(recent_matches.itertuples(), weights)
    )



    # Normalize the weighted score by the sum of weights
    return round(weighted_score / sum(weights), 3) if weights else 0


In [1032]:
df['team_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Team_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)
df['opp_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Opp_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)

In [1033]:
def team_form(df, row, perspective):
    # Determine the team ID based on the perspective ('Team' or 'Opp')
    if perspective == 'Team':
        team_id = row['Team_ID']
    elif perspective == 'Opp':
        team_id = row['Opp_ID']
    else:
        raise ValueError("Perspective must be 'Team' or 'Opp'")
    
    # Get the current match date
    current_date = row['Date_temp']
    
    # Filter past matches for the team
    past_matches = df[((df['Team_ID'] == team_id) | (df['Opp_ID'] == team_id)) &
                      (df['Date_temp'] < current_date)].sort_values(by='Date_temp', ascending=False).head(5)
    
    # Initialize points
    points = 0
    
    # Weights for the matches (most recent match has the highest weight)
    weights = [5, 4, 3, 2, 1]
    
    # Calculate points with weights
    weighted_points_sum = 0
    total_weights = sum(weights[:len(past_matches)])  # Adjust the total weight in case of less than 5 matches
    
    for match, weight in zip(past_matches.itertuples(), weights):
        if (match.Team_ID == team_id and match.FTR == 'H') or (match.Opp_ID == team_id and match.FTR == 'A'):
            points += 3
        elif match.FTR == 'D':
            points += 1
        else:
            points += 0

        weighted_points_sum += points * weight
    
    if total_weights > 0:

        team_form = round(weighted_points_sum / total_weights, 2)

        return team_form
    else:
        return 0  # Return 0 if no past matches found

In [1034]:
df['team_form'] = df.apply(lambda x: team_form(df, x, 'Team') if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)
df['opp_form'] = df.apply(lambda x: team_form(df, x, 'Opp') if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)

In [1035]:
def rolling_avgs_combined(df, row, perspective):
    # Determine the team ID based on the perspective ('Team' or 'Opp')
    if perspective == 'Team':
        team_id = row['Team_ID']
    elif perspective == 'Opp':
        team_id = row['Opp_ID']
    else:
        raise ValueError("Perspective must be 'Team' or 'Opp'")
    
    # Get the current match date
    current_date = row['Date_temp']
    
    # Filter past 5 matches for the team
    past_matches = df[((df['Team_ID'] == team_id) | (df['Opp_ID'] == team_id)) &
                      (df['Date_temp'] < current_date)].sort_values(by='Date_temp', ascending=False).head(5)
    
    # Weights for the matches (most recent match has the highest weight)
    weights = [5, 4, 3, 2, 1]
    
    # Initialize sums and weighted sums
    shots = []
    shots_target = []
    
    # Determine which columns to use and collect the values
    for match in past_matches.itertuples():
        if match.Team_ID == team_id:
            shots.append(getattr(match, 'HS'))  # Home shots
            shots_target.append(getattr(match, 'HST'))  # Home shots on target
        else:
            shots.append(getattr(match, 'AS'))  # Away shots
            shots_target.append(getattr(match, 'AST'))  # Away shots on target
    
    # Calculate the weighted averages of the values
    weighted_shots = sum(s * w for s, w in zip(shots, weights))
    weighted_shots_target = sum(st * w for st, w in zip(shots_target, weights))
    total_weights = sum(weights[:len(shots)])  # Adjust total weight if there are less than 5 matches
    
    avg_shots = weighted_shots / total_weights if total_weights > 0 else 0
    avg_shots_target = weighted_shots_target / total_weights if total_weights > 0 else 0

    # Round the averages to 2 decimal places
    avg_shots = round(avg_shots, 2)
    avg_shots_target = round(avg_shots_target, 2)
    
    return avg_shots, avg_shots_target

In [1036]:
df['team_shots'], df['team_shots_target'] = zip(*df.apply(lambda x: rolling_avgs_combined(df, x, 'Team') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else (0, 0), axis=1))
df['opp_shots'], df['opp_shots_target'] = zip(*df.apply(lambda x: rolling_avgs_combined(df, x, 'Opp') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else (0, 0), axis=1))

In [1037]:
# Function: Average games played in the last 50 days

from datetime import timedelta

def avg_games_played(df, row, team_column):
    team = row[team_column]
    # Ensure current_match_date is a Timestamp for comparison
    current_match_date = pd.to_datetime(row['Date'], dayfirst=True)  # Assuming 'Date' format is 'dd/mm/yy'

    delta = 60
    start_date = current_match_date - timedelta(days=delta)

    # Ensure 'Date' column is in datetime format for comparison
    #df['Date_temp'] = pd.to_datetime(df['Date'], dayfirst=True)  # Convert 'Date' column to datetime if not already done

    # Filter the DataFrame for matches within the last 30 days
    if team_column == 'Team_ID':
        past_matches = df[((df[team_column] == team) | (df['Opp_ID'] == team)) &
                          (df['Date'] >= start_date) & (df['Date'] < current_match_date)]
    else:
        past_matches = df[((df['Team_ID'] == team) | (df[team_column] == team)) &
                          (df['Date'] >= start_date) & (df['Date'] < current_match_date)]

    # If no matches were played in the last 30 days
    if past_matches.empty:
        return 0

    # Calculate weights based on the recency of each match
    weights = (current_match_date - past_matches['Date']).dt.days
    weighted_count = sum(delta - weights + 1)  # '+ 1' to include the match day in the weight

    # Normalize weights to sum to 1 and calculate the weighted average
    total_weight = sum(delta - weights + 1)
    weighted_avg = weighted_count / total_weight

    return weighted_avg


In [1038]:
# Apply the function for each team and opponent
#df['team_avg_games'] = df.apply(lambda x: avg_games_played(df, x, 'Team_ID') 
#    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)
#df['opp_avg_games'] = df.apply(lambda x: avg_games_played(df, x, 'Opp_ID') 
#    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)

In [1039]:
def avg_goals(df, row, team_column):
    # Season and date of the current match
    current_season = row['Season']
    current_date = row['Date']

    # Determine the columns for goals scored and conceded based on perspective
    if team_column == 'Team_ID':
        goals_scored_column = 'FTHG'  # Assuming FTHG is the column for home team goals
        goals_conceded_column = 'FTAG'  # Assuming FTAG is the column for away team goals
    else:
        goals_scored_column = 'FTAG'  # Flip the columns if we are looking from the opponent's perspective
        goals_conceded_column = 'FTHG'

    # Filter matches from the same season and before the current date
    past_matches = df[
        (df['Season'] == current_season) & 
        (df['Date'] < current_date) & 
        ((df['Team_ID'] == row[team_column]) | (df['Opp_ID'] == row[team_column]))
    ]

    # Calculate the average goals scored and conceded
    goals_scored = 0
    goals_conceded = 0
    total_matches = len(past_matches)

    for match in past_matches.itertuples():
        if getattr(match, 'Team_ID') == row[team_column]:  # Team is playing at home
            goals_scored += getattr(match, goals_scored_column)
            goals_conceded += getattr(match, goals_conceded_column)
        else:  # Team is playing away
            goals_scored += getattr(match, goals_scored_column)
            goals_conceded += getattr(match, goals_conceded_column)

    avg_goals_for = goals_scored / total_matches if total_matches > 0 else 0
    avg_goals_against = goals_conceded / total_matches if total_matches > 0 else 0

    # Round the averages to 3 decimal places
    avg_goals_for = round(avg_goals_for, 2)
    avg_goals_against = round(avg_goals_against, 2)

    return avg_goals_for, avg_goals_against


In [1040]:
# Apply the function and create new columns
#df['team_avg_goals_for'], df['team_avg_goals_against'] = zip(*df.apply(lambda x: avg_goals(df, x, 'Team_ID') 
#    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else (0, 0), axis=1))
#df['opp_avg_goals_for'], df['opp_avg_goals_against'] = zip(*df.apply(lambda x: avg_goals(df, x, 'Opp_ID') 
#    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else (0, 0), axis=1))

In [1041]:
# Calculate means only for numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
means = df[numeric_cols].mean()

# Fill missing values in numeric columns with their respective means
df[numeric_cols] = df[numeric_cols].fillna(means)

In [1042]:
# Set the FTR to 'X' where the value is currently NaN
df['FTR'] = df['FTR'].fillna('X')

In [1043]:
# Drop every row where 'FTR' is not 'H', 'D', or 'A', or 'X' (if future matches are included)
df = df[df['FTR'].isin(['H', 'D', 'A', 'X'])]

# Map 'H', 'D', and 'A' to 0, 1, and 2 respectively
df['FTR'] = df['FTR'].map({'H': 0, 'D': 1, 'A': 2, 'X': -1}).astype(int)

In [1044]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Modify data preparation to include odds
def prepare_data(df, features, predict=False):
    if 'HST' in df.columns and 'AST' in df.columns and not predict:
        df['HST'].fillna(df['HST'].mean(), inplace=True)  
        df['AST'].fillna(df['AST'].mean(), inplace=True)
    X = df[features].fillna(0)  
    return X

# Features to include in the model
features = [
    'team_elo', 'opp_elo', 'team_hist_vs', 'opp_hist_vs', 'team_form', 'opp_form',
    'HST', 'AST', 'AvgH', 'AvgA'  
]

# Train the model using historical data where goals are known
historical_data = df.dropna(subset=['FTHG', 'FTAG'])
X_train = prepare_data(historical_data, features)
y_train = historical_data[['FTHG', 'FTAG']]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prediction function to be used for the entire dataset
def predict_xg(df, features):
    X = prepare_data(df, features, predict=True)
    predicted_goals = model.predict(X)
    return predicted_goals[:, 0], predicted_goals[:, 1]  # xG home, xG away

# Predict xG and xGA for all matches in the dataframe
df['team_xg'], df['opp_xg'] = predict_xg(df, features)



In [1045]:
df.shape

(16533, 126)

In [1046]:
df = df[[
    
        'Div', 'Season', 'Date_temp', 'Time', 'DayOTW', 'Team_ID', 'Opp_ID', 'FTR',

        'team_elo', 'opp_elo',

        'team_xg', 'opp_xg',
        
        'team_hist_vs', 
        'opp_hist_vs',

        'team_points',
        'opp_points',
        
        'team_form', 
        'opp_form',

        #'team_avg_goals_for', 
        #'team_avg_goals_against',
        #'opp_avg_goals_for',
        #'opp_avg_goals_against',
         
        'team_shots', 'opp_shots',
        'team_shots_target', 'opp_shots_target',

        #'team_avg_games', 'opp_avg_games',

        'AvgH', 'AvgD', 'AvgA'
         
         
         ]]

In [1047]:
df.head()

,Div,Season,Date_temp,Time,DayOTW,Team_ID,Opp_ID,FTR,team_elo,opp_elo,...,opp_points,team_form,opp_form,team_shots,opp_shots,team_shots_target,opp_shots_target,AvgH,AvgD,AvgA
13535,14,2021,20200801,12,5,3,320,2,1499,1541,...,0,0.0,0.0,0.0,0.0,0.0,0.0,7.11,4.49,1.43
13536,14,2021,20200801,15,5,121,364,1,1459,1522,...,0,0.0,0.0,0.0,0.0,0.0,0.0,2.66,3.19,2.65
13537,14,2021,20200801,15,5,189,209,0,1486,1493,...,0,0.0,0.0,0.0,0.0,0.0,0.0,2.09,3.37,3.43
13538,14,2021,20200801,15,5,365,231,0,1523,1429,...,0,0.0,0.0,0.0,0.0,0.0,0.0,2.89,3.19,2.45
13539,14,2021,20200802,16,6,88,178,0,1589,1485,...,0,0.0,0.0,0.0,0.0,0.0,0.0,1.09,9.93,23.56


In [1048]:
# Print the value counts of the Date_temp column where FTR is -1
print(df[df['FTR'] == -1]['Date_temp'].value_counts())

Date_temp
20240420    93
20240421    45
20240419    14
20240422     7
Name: count, dtype: int64


In [1049]:
# Rename 'Date_temp' to 'Date'
df.rename(columns={'Date_temp': 'Date'}, inplace=True)

In [1050]:
import pandas as pd

try:
   
    if dataprep_start_date is not None:
        # Convert date columns to datetime
        df['Date_temp'] = pd.to_datetime(df['Date'], format='%Y%m%d')
        
        # Filter new data based on start date
        df_new = df[df['Date_temp'] >= dataprep_start_date].copy()

        # Load existing data
        df_existing = pd.read_csv(f'data/processed/processed_data_{content}.csv')

        df_existing['Date_temp'] = pd.to_datetime(df_existing['Date'])
        
        # Filter existing data to remove overlap with new data
        df_existing = df_existing[df_existing['Date_temp'] < dataprep_start_date]

        # Combine and sort data
        df_final = pd.concat([df_existing, df_new], ignore_index=True)
        df_final.sort_values(['Date_temp', 'Time'], inplace=True)

        # Clean up temporary columns
        df_final.drop(columns='Date_temp', inplace=True)
    else:
        df_final = df.copy()

    # Save the final DataFrame
    df_final.to_csv(f'data/processed/processed_data_{content}.csv', index=False)
    print(f"Data saved: {df_final.shape[0]} matches")

except Exception as e:
    print(f"Error: {e}")


Data saved: 16533 matches


In [1051]:
import winsound
frequency = 400  # Set Frequency To 2500 Hertz
duration = 200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)